<a href="https://colab.research.google.com/github/nvaikunt/PromptBasedReranking/blob/main/BaselineTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/nvaikunt/PromptBasedReranking.git

Cloning into 'PromptBasedReranking'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 48 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:30
🔁 Restarting kernel...


In [2]:
!conda --version

conda 4.14.0


In [3]:
%cd PromptBasedReranking/

/content/PromptBasedReranking


In [4]:
!bash setup.sh

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [5]:
!python Baseline/download_dpr_data.py --resource data.wikipedia-split.psgs_w100

Requested resource from %s https://www.dropbox.com/s/bezryc9win2bha1/psgs_w100.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tsv

Downloaded to %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
 Saved to %s ./downloads/data/wikipedia-split


In [6]:
!python Baseline/download_dpr_data.py --resource data.retriever-outputs.dpr.nq-train

Requested resource from %s https://www.dropbox.com/s/6g4erof4ifg8xea/nq-train.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.json

Downloaded to %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.tar.gz
 Saved to %s ./downloads/data/retriever-outputs/dpr


In [7]:

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
import tokenizers
import datasets 
import pandas

wikipedia_txt = datasets.load_dataset("csv", data_files='downloads/data/wikipedia-split/psgs_w100.tsv', delimiter='\t')


Using custom data configuration default-cb3c6fef8f5e82e0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cb3c6fef8f5e82e0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
wikipedia_txt

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'title'],
        num_rows: 21015324
    })
})

In [9]:
nq_open = datasets.load_dataset("json",data_files="downloads/data/retriever-outputs/dpr/nq-train.json")

Using custom data configuration default-a7ea593b34a6dc21


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
nq_open["train"][2]["question"]

'which part of earth is covered with water'

In [11]:
def get_top_k_pos(row, k, txt_database):
  ctxs = row["ctxs"]
  top_k = []
  for ctx in ctxs:
    if ctx["has_answer"]:
      text = txt_database[ctx["id"] - 1]["text"]
      top_k.append((text, "true"))
    if len(top_k) == k:
      break
  if len(top_k) == 0: 
    return []
  while len(top_k) < k:
    top_k.extend(top_k[:(k - len(top_k))])

  return top_k[:k]
  
def get_top_k_pos_neg(row, k, txt_database):
  ctxs = row["ctxs"]
  top_k_pos = []
  top_k_neg = []
  for ctx in ctxs:
    if ctx["has_answer"] and len(top_k_pos) < k:
      text = txt_database[ctx["id"] - 1]["text"]
      top_k_pos.append((text, "true"))
    if not ctx["has_answer"] and len(top_k_neg) < k: 
      text = txt_database[ctx["id"] - 1]["text"]
      top_k_neg.append((text, "false"))
    if len(top_k_pos) == k and len(top_k_neg):
      break 
  if len(top_k_pos) == 0: 
    return []
  while len(top_k_pos) < k:
    top_k_pos.extend(top_k_pos[:(k - len(top_k_pos))])
  while len(top_k_neg) < k:
    top_k_neg.extend(top_k_neg[:(k - len(top_k_neg))])
 
  return top_k_pos + top_k_neg



In [18]:
def create_pos_txt_col(example, k, txt_database):
  return {"pos_text": get_top_k_pos(example, k, txt_database)}

def create_pos_neg_txt_col(example, k, txt_database):
  return {"pos_neg_text": get_top_k_pos_neg(example, k, txt_database)}


In [15]:
from functools import partial
nq_open["train"] = nq_open["train"].map(partial(create_pos_txt_col, k=20, txt_database=wikipedia_txt["train"]), num_proc=4)

#0:   0%|          | 0/19792 [00:00<?, ?ex/s]

#1:   0%|          | 0/19792 [00:00<?, ?ex/s]

#3:   0%|          | 0/19792 [00:00<?, ?ex/s]

#2:   0%|          | 0/19792 [00:00<?, ?ex/s]

In [19]:
nq_open["train"] = nq_open["train"].map(partial(create_pos_neg_txt_col, k=10, txt_database=wikipedia_txt["train"]), num_proc=4)

Process ForkPoolWorker-22:
Process ForkPoolWorker-25:
Process ForkPoolWorker-24:
Process ForkPoolWorker-23:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/s

KeyboardInterrupt: ignored

In [21]:
nq_open["train"][0]["pos_text"]

[['both did it in 2003. The Lions also made the playoffs in 1993, 1994, 1995, 1997 and 1999, making the 1990s one of the most successful decades in team history. In 1993, they went 10–6, first in the NFC Central Division, but lost to the Green Bay Packers. In 1994, they lost to the Packers in the playoffs again. In 1995, the Pittsburgh Steelers lost to the Packers in the final week of the season causing Detroit having to go onto the road to Philly. They lost to the Philadelphia Eagles and former Lion Rodney Peete, in an embarrassing fashion,',
  'true'],
 ["didn't have a winning record in the years spanning 1993 to 2012. The Tigers' best record over this span was 79–83, recorded in 1997 and 2000. In 1998, the Tigers moved from the AL East, where they had been since the divisions were created in 1969, to the AL Central as part of a realignment necessitated by the addition of the expansion Tampa Bay Devil Rays. In 2000, the team left Tiger Stadium in favor of Comerica Park. Soon after it

In [23]:
from tqdm import tqdm
def create_ranking_loss_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_neg_text"]
    question = dataset[i]["question"]
    current_inputs = [f"Question: {question} Passage: {text[0]} Relevant: " for text in texts]
    current_targets = [text[1] for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
  k = [len(targets)/(n*2)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos_neg": k}

def create_q_gen_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_text"]
    question = dataset[i]["question"]
    current_inputs = [f"Passage: {text[0]} Please write a question based on this passage" for text in texts]
    current_targets = [question for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
  k = [len(targets)/(n)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos": k}

def create_q_gen_ranking_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_neg_text"]
    question = dataset[i]["question"]
    current_inputs = [f"Passage: {text[0]} Please write a question based on this passage" for text in texts]
    current_targets = [question for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
  k = [len(targets)/(n*2)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos_neg": k}



In [24]:
training_dict = create_q_gen_baseline_examples(nq_open["train"])

100%|██████████| 79168/79168 [15:36<00:00, 84.56it/s]


In [25]:
q_gen_dataset = datasets.Dataset.from_dict(training_dict)

In [26]:
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

model_checkpoint = "google/t5-base-lm-adapt"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [27]:
q_gen_dataset[3]

{'inputs': 'Passage: responded with an 85–77 season in 1993, but it would be their last winning season for a number of years. On October 2, 1995, manager Sparky Anderson chose to not only end his career with the Tigers, but retire from baseball altogether. From 1994 to 2005, the Tigers did not post a winning record, the longest sub-.500 stretch in franchise history. In 1996, the Tigers lost a then-team record 109 games, under new general manager Randy Smith. The only team in the majors to have a longer stretch without a winning season during this time were the Pittsburgh Pirates, who Please write a question based on this passage',
 'targets': 'when was the last time the detroit lions won a championship',
 'k_pos': 18.17274656426839}

In [28]:
def preprocess_function(examples, max_input_length, max_target_length):
    model_inputs = tokenizer(
        examples["inputs"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(text_target=examples["targets"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels_mask"] = labels["attention_mask"]
    return model_inputs

'which part of earth is covered with water</s>'